# The Interpreter
In this chapter, we will explore the process of building an interpreter for our
testing framework. The interpreter we build will give our tests more power than
they currently have with the text-based RVTs (Requirements Verification Tests) that
we use. The interpreter will be based on Lisp, and specifically influenced by the
Clojure language.

## Rationale
You may be wondering, "Why not just use Python instead of building an interpreter
with Lisp?" It's a fair question. However, one of the original goals of RVTs was to
provide a test procedure that was easy to read. This meant that there was a goal of
making the block syntax read like an English sentence. While we acknowledge that RVTs
did not entirely meet this goal, we believe that it is still an important vision to
strive for.

However, we also recognize that we must be careful in designing our syntax to
ensure that it is as simple as possible. This is where Lisp's expressive power comes
in. Lisp is one of the most expressive programming languages available and has a
rich history of being used for symbolic manipulation and artificial intelligence.
Its syntax is simple and consistent, making it well-suited for expressing complex
predicates and logical statements. The use of Lisp will enable us to create a DSL
(Domain-Specific Language) that is both powerful and easy to read, making it a
natural fit for our testing framework.

Here is an example of verifying if a connector is equal to a value using our text-based RVT syntax:

```bash
Verify SOME.COMPONENT.MEMBER eq 10
```

And here is the equivalent syntax in our Lisp-based interpreter:

```lisp
(Verify SOME.COMPONENT.MEMBER = 10)
```

At first glance, the two examples look similar and one might even argue that the
text-based RVT syntax reads better. However, let's consider a slightly more complex
example where we need to verify if a connector is greater than x and less than y.
The text-based RVT syntax would require two separate verification lines:

```bash
Verify SOME.COMPONENT.MEMBER gt 10
Verify SOME.COMPONENT.MEMBER lt 100
```

On the other hand, the Lisp-based syntax allows us to use arbitrary predicate
functions to express complex verification requirements in a more concise and
readable way:

```lisp
(Verify SOME.COMPONENT.MEMBER (fn [x] (< 10 x 100)))
```

While the Lisp-based syntax may seem a little more complex, the ability to use
arbitrary predicate functions is extremely useful. In contrast, the text-based RVTs
only support a limited set of functions, such as `eq`, `neq`, `gt`, `gte`, `lt`,
and `lte`. If anyone wants to add another function to the RVTs, they would need
to modify the test framework code, which is not ideal. In practice, this can
discourage users from adding new functions to the RVTs.

In addition to the flexibility of using arbitrary predicate functions, the
Lisp-based syntax allows us to name these functions to make them more readable.
For example, we can define a function called is-in-valid-range as follows:

```lisp
(defn is-in-valid-range [x] (< 10 x 100))
(Verify SOME.COMPONENT.MEMBER is-in-valid-range)
```

This makes it easier for readers of the test script to understand what the
predicate function is doing, which can lead to more maintainable and extensible
tests.


## Syntax Overview
Lisp syntax is quite different from C/C++ syntax, but it has its own unique 
advantages. Lisp is a functional programming language, which means that it 
treats computation as the evaluation of mathematical functions and avoids 
changing state and mutable data. Here are some key features of Lisp syntax 
that distinguish it from C/C++:

1. S-expressions: Lisp syntax is based on S-expressions, which are nested 
   lists of symbols and data. S-expressions are the primary means of expressing 
   Lisp code.
2. Prefix notation: Lisp expressions are written in prefix notation, which 
   means that the operator comes before the operands. For example, instead of
   writing "2 + 3", you would write "(+ 2 3)".
3. No operator precedence: In Lisp, all operators have the same precedence, 
   so parentheses are used to group expressions in the desired order. For 
   example, to evaluate "(+ 2 (* 3 4))", the multiplication operation is 
   evaluated first, then the addition operation.
4. Anonymous functions: Lisp supports anonymous functions, which are functions 
   without a name. This is useful for passing functions as arguments to other 
   functions or for defining short, throwaway functions.
5. Macros: Lisp supports macros, which are a way to programmatically generate 
   code. Macros can be used to abstract away boilerplate code, to generate 
   repetitive code, or to create domain-specific languages.

Overall, Lisp syntax may take some getting used to, but its functional programming
features make it a powerful and flexible language.

## Environment
In order to manage symbols and function calls, we need an environment. The 
environment will be a dictionary-like object that allows us to store and retrieve 
values by their associated keys. However, our environment will have one key 
difference: it will allow us to nest an outer environment. This outer environment
will only be used for lookups, and will allow us to modify the inner environment 
without affecting the outer environment.

Our first implementation of an environment will be simple, and will allow us to 
bind a list of parameter symbols to a list of arguments. To achieve this, we have 
created a destructure method that allows us to define environments with more complex 
destructuring rules than what we are starting with. By using destructure, we can
more easily extract and bind values from nested data structures, such as lists or 
maps, to symbols within our environment.

As we build our Lisp interpreter, we will need to continue expanding our environment
to support more complex operations and functions. However, our initial implementation
will allow us to start building a functional testing DSL that can take advantage of
Lisp's expressive syntax and powerful features

In [ ]:
import nbloader
nbloader.install_loader()

from nbdev import patch

utils = __import__('Appendix - B - Utilities')
reader = __import__('02 - The Reader')

read = reader.read
Symbol = reader.Symbol
List = reader.List

In [ ]:
class Env(dict):
    def __init__(self, params=(), args=(), outer=None):
        self.outer = outer
        self.destructure(params, args)
        
    def destructure(self, params, args):
        if len(params) != len(args):
            raise TypeError(f"Invalid arguments[{args}] received. Expected [{params}]")
        self.update(zip(params, args))
    
    def __contains__(self, key):
        return super().__contains__(key) or key in self.outer
    
    def __getitem__(self, key):
        if super().__contains__(key):
            return super().__getitem__(key)
        elif self.outer is not None and key in self.outer:
            return self.outer[key]
        else:
            raise KeyError(f'{key} not found.')

In [ ]:
# Test
# use the reader for our params and arguments

a = Symbol('a')
b = Symbol('b')
x = Symbol('x')
y = Symbol('y')
test = Symbol('test')

outer = Env(params=read('[test a b]'), args=read('[1 2 3]'))
env = Env(params=read('[x y test]'), args=read('[4 5 6]'), outer=outer)

# inner 1
assert outer['a'] == 2
assert outer[b] == 3

# inner 2
assert env[x] == 4
assert env[y] == 5

# outer
assert env[a] == 2
assert env[b] == 3


# outer with overwrite
assert env[test] == 6
assert outer[test] == 1

### Global Environment
Our interpreter will by default start with an enviroment that is already
loaded with a set of functions. This can be extended over time.

In [ ]:
import math
import operator as op

def standard_env():
    env = Env()
    
    # math functions
    env.update({k:v 
                for k,v in vars(math).items() 
                if not k.startswith('__')})
    
    env.update({
        '+':op.add, '-':op.sub, '*':op.mul, '/':op.truediv, 
        '>':op.gt, '<':op.lt, '>=':op.ge, '<=':op.le, '=':op.eq, 
        'abs':     abs,
        'append':  op.add,  
        'apply':   lambda proc, args: proc(*args),
        'begin':   lambda *x: x[-1],
        'car':     lambda x: x[0],
        'first':   lambda x: x[0],
        'cdr':     lambda x: x[1:],
        'rest':    lambda x: x[1:],
        'cons':    lambda x,y: [x] + y,
        'eq?':     op.is_, 
        'expt':    pow,
        '=':       op.eq, 
        'count':   len, 
        'list':    lambda *x: reader.List(x), 
        'list?':   lambda x: isinstance(x, reader.List), 
        'map':     map,
        'max':     max,
        'min':     min,
        'not':     op.not_,
        'null?':   lambda x: x == [], 
        'number?': lambda x: isinstance(x, (int, float)),  
        'print':   print,
        'procedure?': callable,
        'round':   round,
        'symbol?': lambda x: isinstance(x, reader.Symbol),
    })
    return env

global_env = standard_env()

## Eval
The eval function is responsible for taking in a data structure and evaluating 
expressions. It has some very simple rules:

1. If the expression is a `Symbol`, the `eval` function looks up the symbol in the 
   current environment and returns the associated value.
2. If the expression is not a `List` it is returned as is.
3. If the expression is a `List` and the first argument is a `special form`
   then the special form is used to evalute the expression. Special forms are 
   a set of built-in functions in Lisp that have special evaluation rules. 
   Examples include `if`, `let`, and `defn`. 
4. If the expression is a `List` and the first argument has been registered as a
   `macro` then the registered function for that macro is used to operate on the
   expression. Macros are functions that operate on Lisp code at compile-time and
   generate new code to be evaluated later.
5. Finally if the expression is a `List` the first argument is called as a function
   with the rest of the expression as its arguments


In [ ]:
# export

special_forms = {}

def eval(x, env=global_env):
    "Evaluate an expression in an environment."

    # symbol reference
    if isinstance(x, reader.Symbol):
        return env[x]
    
    # constant
    elif not isinstance(x, reader.List):
        return x 
    
    # special forms
    elif x[0] in special_forms:
        return special_forms[x[0]](x, env)

    # procedure call
    else:
        proc = eval(x[0], env)
        args = [eval(arg, env) for arg in x[1:]]
        return proc(*args)

In [ ]:
# Helper
def readeval(s):
    return eval(read(s))

readeval('(print "Hello World")')

## Special Forms
The above eval function is missing any implementation of special forms.
Special forms are special cases of syntax used in the evaluation of 
expressions. They are special in that they are not evaluated like regular 
function calls but instead are evaluated directly by the Lisp evaluator. 
Special forms are the primitive building block of lisp. Everything else
can be built using them.

### (if test then else?)
The `if` special form that takes three arguments: `test`, `then`, `else`. 
It evaluates the first argument and if it evaluates to `false` or `nil` the 
`else` expession is evaluated and return otherwise the `then` expression is
evaluated and returned

In [ ]:
# export

def if_special_form(x, env):
    (_, test, then, _else) = x if len(x) == 4 else x + [None]
    exp = (then if eval(test, env) else _else)
    return eval(exp, env)
special_forms['if'] = if_special_form

In [ ]:
# test

assert readeval('(if true 1)') == 1
assert readeval('(if false 1)') == None
assert readeval('(if true 1 2)') == 1
assert readeval('(if false 1 2)') == 2

### (do exprs*)
Evaluates the expressions `exprs` in order and returns the value of the last. If no expressions are supplied, returns `nil`.


In [ ]:
# export

def do_special_form(x, env):
    _, *expressions = x
    last = None
    for exp in expressions:
        last = eval(exp, env)
    return last
special_forms['do'] = do_special_form

In [ ]:
assert readeval('(do 1 2)') == 2
assert readeval('(do)') == None

### def


In [ ]:
# export

def def_special_form(x, env):
    (_, symbol, exp) = x
    env[symbol] = eval(exp, env)
special_forms['def'] = def_special_form

In [ ]:
# test

assert readeval('''
(do 
    (def x 1)
    x)
''') == 1

### (let [ binding* ] expr*)
Evaluates the expressions exprs in a lexical context in which the symbols 
in the binding-forms are bound to their respective init-exprs or parts 
therein. The bindings are sequential, so each binding can see the prior 
bindings. The exprs are contained in an implicit do. If a binding symbol is
annotated with a metadata tag, the compiler will try to resolve the tag to a
class name and presume that type in subsequent references to the binding.

In [ ]:
# export

def let_special_form(x, env):
    _, bindings, *exprs = x
    env = Env(outer=env)
    # binding to environment
    for binding, expr in zip(bindings[::2], bindings[1::2]):
        env[binding] = eval(expr, env)
    return eval(List(['do'] + exprs), env)
special_forms['let'] = let_special_form

In [ ]:
# test

assert readeval('''
(let [x 2
      y x]
   2)
''') == 2

### (quote form)
Returns form unevaluated

In [ ]:
# export

def quote_special_form(x, env):
    _, form = x
    return form
special_forms['quote'] = quote_special_form


In [ ]:
# test

assert readeval("'(1 2 3)") == [1, 2, 3]

### (fn name? [ params* ] exprs*)

In [ ]:
# export

def create_function(name, params, exprs, env):
    def fn(*args):
        return eval(List([Symbol('do'),  *exprs]), Env(params, args, outer=env))
    if name is not None:
        fn.__name__ = name
    return fn

def fn_special_form(x, env):
    _, *args = x
    name, params, *exprs = args if len(args) >= 3 else [None] + args
    return create_function(name, params, exprs, env)
special_forms['fn'] = fn_special_form
    

In [ ]:
# test

readeval('''
(do
    (def add2 (fn test [a b] (+ a b)))
    (add2 4 5))
''')